In [1]:
from wayne_utils import load_data, save_data
import os
import sys
import pandas as pd
import shutil
from tools.rule_split import cpl_text_ruled_processor


_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/text2table_preprocess/CPL"
sys.path.insert( 0, os.path.join( _ROOT_PATH, "raw/CPL") )

table_path = os.path.join( _ROOT_PATH, "pairs/tables.json" )
text_path = os.path.join( _ROOT_PATH, "pairs/texts.text" )

# 规则处理texts：按划分等级形成三个json并拷贝texts.text到data

In [67]:
cpl_text_ruled_processor()
shutil.copy( text_path, os.path.join( _ROOT_PATH, "data/ruled_texts/texts.text" ))
used_texts = {}
texts_splited_C = load_data( os.path.join( _ROOT_PATH, "data/ruled_texts/texts_splited_C.json" ) , "json")
for i in range( len( texts_splited_C ) ):
    temp = {}
    temp[ '开头' ] = texts_splited_C[i][ '开头' ]
    temp[ '程序信息' ] = texts_splited_C[i][ '正文' ][ '程序信息' ]
    temp[ '原告申诉' ] = texts_splited_C[i][ '正文' ][ '申辩证' ][ '原告' ]
    temp[ '法院裁定' ] = texts_splited_C[i][ '正文' ][ '申辩证' ][ '法院' ]
    used_texts[i] = temp

save_data( used_texts, os.path.join( _ROOT_PATH, "data/texts.json" ))

# 规则处理table：根据可用字段筛选出元组字典作为标签，简单集仅包含entity和原告诉请、法院裁决20个字段

In [3]:
def get_entity( table ):
    """返回entity三元组"""
    entity_dict = table[ "first_column" ]
    temp = set()
    for key in entity_dict:
        if key == "法院":
            temp.add( ( "法院", "姓名名称", entity_dict[ key ][0] ) )
        elif key == "出借人（原告）":
            temp.add( ( "原告", "姓名名称", entity_dict[ key ][0] ) )
        elif "被告" in key:
            temp.add( ( "被告", "姓名名称", entity_dict[ key ][0] ) )
    return temp

def get_datacell( table ):
    """返回Actually_used_attributes指定字段的三元组"""
    temp = set()
    datacell = table["data_cell"]
    for key in datacell:
        if key in Dynamic_Simple:
            value = datacell[key]["出借人（原告）"]['取值']
            if 'nan' != value:
                temp.add( ( "原告", key, value) )
            value = datacell[key]["法院"]['取值']
            if 'nan' != value:
                temp.add( ( "法院", key, value) )
    return temp

In [4]:
tables = load_data( table_path, "json")
label_list = []
for i in range(len(tables)):
    table = tables[str(i)]
    entities = get_entity( table )              # 仅仅识别法院与原告姓名
    datacell = get_datacell( table )            # 仅识别法院申诉的三个请求
    label_list.append( entities | datacell)

In [7]:
tables[str(0)]

{'first_column': {'法院': ['安徽省合肥市包河区人民法院', 2],
  '出借人（原告）': ['卢江平', 5],
  '借款人（被告）': ['刘维辉', 8]},
 'data_cell': {'是否变更诉讼请求': {'法院': {'原文': 'nan', '取值': 'nan'},
   '出借人（原告）': {'原文': 'nan', '取值': 'nan'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回本金总额（元）': {'法院': {'原文': '被告刘维辉借原告卢江平人民币707100元', '取值': '707100'},
   '出借人（原告）': {'原文': '请求法院判令被告向原告偿还借款707100元', '取值': '707100'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回利息总额（元）': {'法院': {'原文': 'nan', '取值': 'nan'},
   '出借人（原告）': {'原文': 'nan', '取值': 'nan'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回利息计算起始日期': {'法院': {'原文': 'nan', '取值': 'nan'},
   '出借人（原告）': {'原文': 'nan', '取值': 'nan'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回利息计算起始日期类型': {'法院': {'原文': 'nan', '取值': 'nan'},
   '出借人（原告）': {'原文': 'nan', '取值': 'nan'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回利息计算截止日期': {'法院': {'原文': 'nan', '取值': 'nan'},
   '出借人（原告）': {'原文': 'nan', '取值': 'nan'},
   '借款人（被告）': {'原文': 'nan', '取值': 'nan'}},
  '需返回利息计算截止日期类型': {'法院': {'

In [12]:
save_data( label_list, os.path.join( _ROOT_PATH, "data/tables.pickle" ))

# 处理完整的字段：Dynamic Simple

In [2]:
from tools.cpl_level import Dynamic_Simple

table_total_path = os.path.join( _ROOT_PATH, "pairs/tables_total.json" )
table_total = load_data( table_total_path, "json")

In [ ]:
# 抽取实体
def entity_extr( entity_dict ):
    entities = {
        "法院": [],
        "原告": [],
        "被告": [],
    }
    for key in entity_dict:
        if key == "法院":
            entities["法院"].append( entity_dict[key][0] )
        elif "出借人（原告）" in key:
            entities["原告"].append( entity_dict[key][0] )
        elif "借款人（被告）" in key:
            entities["被告"].append( entity_dict[key][0] )
        else:
            raise Exception(f"Key {key} 不存在")
    return entities

# 抽取truth属性
dynamic_key = {
    "借款次数": int,                                      # '借款次数'和后面次数约定有强相关吗
    '是否有借款凭证': bool,
    "借款凭证":{
        "名称": str,
        "所载内容": str,
    },
    "约定的借款金额": {
        "约定情况": ['口头约定', '书面约定', '未约定'],     # '未约定'出现在借款次数中但没有约定的情况，子字段可以不填
        "实际发生时间": str,
        "金额（元）": str,
    },
    "约定的还款日期或借款期限": {
        "约定情况": ['口头约定', '书面约定', '未约定'],
        "实际发生时间": str,
        "还款日期": str,
    },
    "约定的利息": {
        "约定情况": ['口头约定', '书面约定', '未约定'],
        "实际发生时间": str,
        "利率数值（百分比或元）": str,
    },
    "约定的逾期利息": {
        "约定情况": ['口头约定', '书面约定', '未约定'],
        "实际发生时间": str,
        "利率数值（百分比或元）": str,
    },
    "约定的违约金": {
        "约定情况": ['口头约定', '书面约定', '未约定'],
        "实际发生时间": str,
        "违约金数值（百分比或元）": str,
    },
}

def multi_ins( entity_type, dicts ):
    ent_name = entity_dict[ entity_type ][0]
    ent_type = None
    types = [ "法院", "原告", "被告"] 
    for _type in types:
        if _type in entity_type:
            ent_type = _type
            break
    value = dicts[entity_type]['取值']
    return ent_type, ent_name, value

In [58]:
len(table_total)

703

In [60]:
ret_dict_all = {}
for index in range( len(table_total)):
    entity_dict = table_total[ str(index) ]['first_column']
    datacell_list = table_total[ str(index) ]['data_cell']
    cell_dict = datacell_list[0]            # key = (['【诉讼请求】【法院认为及判决结果】', '【事实与理由】【辩称】【法院认定事实】', '专题问题'])
    cell_shape = datacell_list[1]

    claims = cell_dict['【诉讼请求】【法院认为及判决结果】']      # ['【诉讼请求原文】【法院认为及判决结果原文】', '是否变更诉讼请求', '需返回本金总额（元）', '需返回利息', '需返回逾期利息', '需返回违约金', '担保责任', '需返回债权实现费用']
    truth = cell_dict['【事实与理由】【辩称】【法院认定事实】']   #　['【事实与理由原文】【辩称原文】【法院认定事实原文】', '借款目的与用途类型', '借款人与出借人关系类型', '担保人的姓名或单位名称', '借款人与担保人关系类型', '意思表示日', '借款次数', '借款次数情况', '是否有借款凭证', '借款凭证', '约定的借款金额', '约定的还款日期或借款期限', '约定的利息', '约定的逾期利息', '约定的违约金', '管辖约定情况', '仲裁约定情况', '程序性费用的承担（律师费、诉讼费等）', '约定的还款方式', '约定的担保', '借款实际交付', '已还款总体情况类型', '已还款', '【当事人提供的所有证据原文】或【反映认定的所有证据原文】']
    special = cell_dict['专题问题']                             # ['夫妻共同债务', '砍头息', '公司为股东担保', '债权/债务非原始取得', '出借人主体资格问题', '职业放贷人', '互联网平台责任', '民刑交叉']
    # 抽取实体
    entities = entity_extr( entity_dict )
    entities

    # 抽取truth属性
    ret_dict = {}
    for key in entities.keys():
        ret_dict[key] = {}
        for ins in entities[key]:
            ret_dict[key][ins] = {}
    # 遍历原始字典的字段
    for key in truth.keys():
        if key in ['借款次数', '是否有借款凭证']:
            temp = truth[ key ]
            for entity_ins in temp:
                ent_type, ent_name, value = multi_ins( entity_ins,  temp)
                try:
                    ret_dict[ent_type][ent_name][key] = value
                except:
                    raise Exception( f"{ent_type, ent_name, value}")
        else:
            if key in dynamic_key.keys():
                dynamic_dict = truth[key]
                if not isinstance( dynamic_dict, dict):
                    raise Exception( f"{dynamic_dict} type incorrect")
                number_keys = dynamic_dict.keys()
                # 遍历动态字段的所有次数
                for num in dynamic_dict.keys():
                    for sub_key in dynamic_dict[ num ]:
                        # 遍历所有次数中的子字段，寻找本次采集目标
                        if sub_key in dynamic_key[key].keys():
                            temp = truth[ key ][num][sub_key]
                            # 如果是dynamic_key中的字段，遍历所有角色实例
                            for entity_ins in temp:
                                ent_type, ent_name, value = multi_ins( entity_ins,  temp)
                                # if value != "nan":
                                if key not in ret_dict[ent_type][ent_name]:
                                    ret_dict[ent_type][ent_name][key] = {}
                                if num not in ret_dict[ent_type][ent_name][key]:
                                    ret_dict[ent_type][ent_name][key][num] = {}
                                ret_dict[ent_type][ent_name][key][num][sub_key] = value
    ret_dict_all[index] = ret_dict

In [66]:
save_data( ret_dict_all, os.path.join(_ROOT_PATH, "data/dynamic_simpe_table.json"))